In [ ]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 1. Huggingface Datasets에서 법 데이터 가져오기

법률 분야는 전문 지식을 필요로 하는 가장 전문적인 분야 중 하나입니다. 자연어 처리에는 다양한 측면이 필요하며, 당사는 데이터 세트 요구 사항에 중점을 두고 있습니다. 신경망 모델의 테스트와 학습을 위해서는 골드 스탠다드가 필요한 만큼, 이번 데이터셋 공개가 법률 분야, 특히 한국 법률 시스템을 위한 자연어 처리의 발전에 도움이 되기를 바랍니다.

https://huggingface.co/datasets/lawcompany/KLAID

한국의 판례 데이터를 기반으로 한 데이터 세트입니다.

In [ ]:
from datasets import load_dataset

datasets = load_dataset("lawcompany/KLAID", 'ljp')

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
print(datasets)

DatasetDict({
    train: Dataset({
        features: ['laws_service_id', 'fact', 'laws_service'],
        num_rows: 161192
    })
})


## 2. 임베딩을 수행하기 위해 Tokenizer, Model 정의

한국어 clustering, semantic search에 자주 쓰이는
RoBERTa 모델을 허깅페이스에서 불러옵니다.

https://huggingface.co/jhgan/ko-sroberta-multitask

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("jhgan/ko-sroberta-multitask")
model = AutoModel.from_pretrained("jhgan/ko-sroberta-multitask")

임베딩 함수를 정의합니다.

objective -> 'fact', 즉 판례를 [1, 768] 크기의 숫자 벡터로 임베딩합니다. <br>
model을 gpu에 할당해 더 빠른 연산을 진행합니다.

In [ ]:
from tqdm.notebook import tqdm

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

def embed_facts(facts):

    inputs = tokenizer(facts, return_tensors="pt", padding=True, truncation=True)
    inputs = inputs.to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()

    return embeddings

model.to(device)

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Drop

In [ ]:
# 판례 10,000개 임베딩 진행

progress_bar = tqdm(total=len(datasets['train'][:10_000]['fact']), desc= "판례 임베딩")

embeddings = []
for fact in datasets['train'][:10_000]['fact']:
    embedding = embed_facts([fact])
    embeddings.append(embedding)
    progress_bar.update(1)

progress_bar.close()

Embedding facts:   0%|          | 0/10000 [00:00<?, ?it/s]

# 3. 코사인 유사도 함수로 가장 유사한 판례 + 판례명 출력

코사인 유사도 함수로 input sentence를 임베딩한 input_embedding [1, 768] 숫자벡터와<br>
[10000, 768] 크기의 10,000 개의 판례 중 가장 벡터값이 유사한 판례를 고른 후 출력합니다.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def find_most_similar(embedding, embeddings): # embedding : 입력 문장, embeddings : 720 크기의 케이스 임베딩들
    ## 전체 임베딩 결과에 대해 코사인 유사도 함수 적용
    similarities = cosine_similarity(embedding.reshape(1, -1), np.array(embeddings).reshape(len(data['fact']), -1))
    ## 가장 유사한 결과 10개 most_similar_index에 할당
    most_similar_index = np.argsort(similarities[0])[- 10:][::-1]

    return most_similar_index

In [ ]:
input_fact = str(input('대조를 원하는 판례를 입력하세요. : '))
input_embedding = embed_facts([input_fact])[0]

most_similar_index = find_most_similar(input_embedding, embeddings)
most_similar_fact = datasets['train']['fact'][most_similar_index]
most_similar_case = datasets['train']['laws_service'][most_similar_index]

print("가장 유사한 판례:", most_similar_fact)
print('가장 유사한 판례명:', most_similar_case)

대조를 원하는 판례를 입력하세요. : 사이버 스토킹으로 인한 고소
가장 유사한 판례: 피고인은 트윗터(www.twitter.com)에서 ‘B’ 라는 아이디를 사용하는 사람이다. 피고인은 2014. 6. 14. 자신의 주거지(또는 장소를 알 수 없는 곳)에서 스마트폰을 이용하여 트윗터 계정에 접속하여, 피해자 C(트윗터 아이디 : D)에게 “C씨 계속 숨어 있제  씨발놈아” 라고 트윗하는 등 별지 범죄일람표와 같이 모두 24회에 걸쳐 피해자를 모욕하였다.
가장 유사한 판례명: 형법 제311조
